### ProjectManager

ProjectManager is used to interact with multiple modules within ORBIT. This class allows any combination of modules to be configured and ran together to represent an entire project in ORBIT. It handles the configuration of each module and maps outputs of design modules into installation modules where necessary. This tutorial goes through how to build up a project level configuration using ProjectManager.

In [2]:
from ORBIT import ProjectManager

In [3]:
# The compile expected configs for multiple modules within ProjectManager, use the 'compile_input_dict' method:
# In this example, we'll configure ProjectManager to run the MonopileDesign and MonopileInstallation modules.

ProjectManager.compile_input_dict(["MonopileDesign", "MonopileInstallation"])

{'wtiv': 'dict | str',
 'feeder': 'dict | str (optional)',
 'num_feeders': 'int (optional)',
 'site': {'depth': 'm', 'distance': 'km', 'mean_windspeed': 'm/s'},
 'plant': {'num_turbines': 'int'},
 'turbine': {'hub_height': 'm',
  'rotor_diameter': 'm',
  'rated_windspeed': 'm/s'},
 'port': {'num_cranes': 'int (optional, default: 1)',
  'monthly_rate': 'USD/mo (optional)',
  'name': 'str (optional)'},
 'monopile_supply_chain': {'enabled': '(optional, default: False)',
  'substructure_delivery_time': 'h (optional, default: 168)',
  'num_substructures_delivered': 'int (optional: default: 1)',
  'substructure_storage': 'int (optional, default: inf)'},
 'monopile_design': {'yield_stress': 'Pa (optional)',
  'load_factor': 'float (optional)',
  'material_factor': 'float (optional)',
  'monopile_density': 'kg/m3 (optional)',
  'monopile_modulus': 'Pa (optional)',
  'monopile_tp_connection_thickness': 'm (optional)',
  'transition_piece_density': 'kg/m3 (optional)',
  'transition_piece_thickne

In [4]:
# For simplicity, we are going to ignore the optional 'monopile_design' and 'project_parameters' subdicts.

config = {
    'wtiv': 'example_wtiv',
    'site': {                   # The inputs required for the design module and
        'depth': 20,            # the installation module are combined into the 'site' subdict
        'distance': 50,
        'mean_windspeed': 9.5
    },

    'plant': {
        'num_turbines': 50
    },

    'turbine': {
        'rotor_diameter': 220,
        'hub_height': 120,
        'rated_windspeed': 13
    },

    # Sizing information for the substructure are not required as they will
    # be calculated by 'MonopileDesign' and passed into 'MonopileInstallation'
    # automatically by 'ProjecManager'.

    # --- Module Definitions ---
    'design_phases': ['MonopileDesign'],
    'install_phases': ['MonopileInstallation'],
}

project = ProjectManager(config)
project.run()

print(f"Total Substructure Cost: {project.system_capex/1e6:.2f} M")
print(f"Total Installation Cost:  {project.installation_capex/1e6:.2f} M")

ORBIT library intialized at '/Users/nriccobo/GitHub/ORBIT/library'
Total Substructure Cost: 258.67 M
Total Installation Cost:  25.67 M


#### Weather

In [5]:
# Weather can be included in the same way as an individual module:
import pandas as pd
weather = pd.read_csv("data/example_weather.csv", parse_dates=['datetime']).set_index("datetime")

project = ProjectManager(config, weather=weather)
project.run()

print(f"Total Substructure Cost: {project.system_capex/1e6:.2f} M")
print(f"Total Installation Cost:  {project.installation_capex/1e6:.2f} M")

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

Total Substructure Cost: 258.67 M
Total Installation Cost:  31.62 M


### Looking inside a module
Look at intermediate variables within a module in two ways.
1. Call the phase of the ProjectManager
2. Run the module on its own and call the variables

These should yield the same results

In [23]:
# 1. Call through ProjectManager
project = ProjectManager(config)
project.run()

print(f"Total Monopile Cost:  ${project.phases['MonopileDesign'].total_cost/1e6:.2f} M")

Total Monopile Cost:  $258.67 M


In [24]:
# 2. Call through the module
from ORBIT.phases.design import MonopileDesign

design = MonopileDesign(config)
design.run()

print(f"Total Monopile Cost:  ${design.total_cost/1e6:.2f} M")

Total Monopile Cost:  $258.67 M
